In [60]:
# Librerias python
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import seaborn as sns


# Importar la librería necesaria para dejar los registros en tablas
from IPython.display import display

In [61]:
# conexion google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **PRoceso para archivo ficticio**

In [62]:
# Ruta del archivo CSV limpio del ejercicio anterior
CSV_ORIGINAL_PATH = '/content/drive/MyDrive/Data_prueba/ventas_challenge_limpio.csv'

# Ruta nueva
CSV_NUEVO_PATH = '/content/drive/MyDrive/Data_prueba/ventas_challenge_limpio_2.csv'

# --- 0.1: Cargar el DataFrame Limpio Original ---
print("--- PASO 0.1: Cargando el archivo CSV original limpio ---\n")
try:
    df_original = pd.read_csv(CSV_ORIGINAL_PATH)
    print(f"'{CSV_ORIGINAL_PATH}' cargado exitosamente. Filas: {df_original.shape[0]}")
except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo '{CSV_ORIGINAL_PATH}'. Asegúrate de que la ruta sea correcta.")
    exit() # Termina el script si el archivo no se encuentra

# --- 0.2: Hacer una identificación de clientes Únicos y Preparar Datos Ficticios para Contactos ---
print("\n--- PASO 0.2: Identificando clientes únicos y preparando datos ficticios ---\n")

# Por cliente unico generación email y nombre por cliente.
# Esto evitará duplicados de contactos si un cliente aparece en varias ventas.
clientes_unicos = df_original['id_cliente'].drop_duplicates().to_frame()
print(f"Clientes únicos encontrados: {clientes_unicos.shape[0]}\n")

# Generación de nombres y correos electrónicos ficticios para cada id_cliente único
# Uso del patrón para que algunos emails sean '@empresa.com' (existentes) y otros '@otra.com' (nuevos)
# Esto simulará la lógica de la "API ficticia".
def generar_datos_contacto_ficticios(row_index, id_cliente):
    # Genera un nombre básico
    nombre_contacto = f"Cliente_{id_cliente[:5]}" # Toma los primeros 5 caracteres del ID para el nombre

    # Alterna entre dominios para simular contactos existentes y nuevos
    if row_index % 2 == 0:
        email_dominio = "@empresa.com" # Simula un contacto existente
    else:
        email_dominio = "@otra.com"    # Simula un contacto nuevo

    # Crea un email pseudo-único usando parte del id_cliente
    email_contacto = f"contacto_{abs(hash(id_cliente)) % 10000}{email_dominio}"
    return nombre_contacto, email_contacto

# Aplicar la función para generar nombre y email para cada cliente único
# Usamos .apply y pasamos el índice de la fila para la lógica del dominio
clientes_unicos[['nombre_contacto', 'email_contacto']] = \
    clientes_unicos.reset_index().apply(lambda row: generar_datos_contacto_ficticios(row.name, row['id_cliente']), axis=1, result_type='expand')

# Eliminar la columna 'index' temporal if it was created with reset_index()
if 'index' in clientes_unicos.columns:
    clientes_unicos = clientes_unicos.drop(columns=['index'])

print("Primeros 5 contactos ficticios generados:\n")
print(clientes_unicos.head())

# --- 0.3: Unir los nuevos campos de contacto al DataFrame original ---
print("\n--- PASO 0.3: Uniendo los nuevos campos al DataFrame de ventas ---\n")

# Unir (merge) los correos y nombres de contacto al DataFrame original de ventas
# Esto asegurará que cada fila de venta tenga el email y nombre de su cliente asociado.
df_preparado = pd.merge(df_original, clientes_unicos[['id_cliente', 'nombre_contacto', 'email_contacto']],
                        on='id_cliente', how='left')

print("Verificando las nuevas columnas en el DataFrame preparado:\n")
print(df_preparado[['id_cliente', 'nombre_contacto', 'email_contacto']].head())
print(f"\nFilas en el DataFrame preparado: {df_preparado.shape[0]}")

# --- 0.4: Guardar el nuevo DataFrame preparado en `ventas_challenge_limpio_2.csv` ---
print(f"\n--- PASO 0.4: Guardando el DataFrame preparado como '{CSV_NUEVO_PATH}' ---\n")

df_preparado.to_csv(CSV_NUEVO_PATH, index=False, decimal='.', sep=',') # Usamos el mismo formato de decimal/separador

print(f"'{CSV_NUEVO_PATH}' guardado exitosamente. Ahora se puede usar este archivo para el Ejercicio 2.")

--- PASO 0.1: Cargando el archivo CSV original limpio ---

'/content/drive/MyDrive/Data_prueba/ventas_challenge_limpio.csv' cargado exitosamente. Filas: 800

--- PASO 0.2: Identificando clientes únicos y preparando datos ficticios ---

Clientes únicos encontrados: 200

Primeros 5 contactos ficticios generados:

  id_cliente nombre_contacto             email_contacto
0      C0094   Cliente_C0094  contacto_2678@empresa.com
1      C0167   Cliente_C0167     contacto_8056@otra.com
2      C0029   Cliente_C0029  contacto_8612@empresa.com
3      C0052   Cliente_C0052     contacto_3517@otra.com
4      C0070   Cliente_C0070  contacto_5563@empresa.com

--- PASO 0.3: Uniendo los nuevos campos al DataFrame de ventas ---

Verificando las nuevas columnas en el DataFrame preparado:

  id_cliente nombre_contacto             email_contacto
0      C0094   Cliente_C0094  contacto_2678@empresa.com
1      C0167   Cliente_C0167     contacto_8056@otra.com
2      C0029   Cliente_C0029  contacto_8612@empresa.co

#**Configuración Inicial y Carga de Datos**

configuración de las rutas de los archivos y carga CSV recién creado.

In [63]:

import json # Necesario para trabajar con formato JSON

# Ruta del archivo CSV preparado para este ejercicio
CSV_PARA_EJERCICIO2_PATH = '/content/drive/MyDrive/Data_prueba/ventas_challenge_limpio_2.csv'
# Ruta donde se guardará el archivo JSON con los nuevos contactos
JSON_NUEVOS_CONTACTOS_PATH = '/content/drive/MyDrive/Data_prueba/nuevos_contactos_a_crear.json'
# Ruta donde se guardará el archivo Excel de resumen
EXCEL_RESUMEN_PATH = '/content/drive/MyDrive/Data_prueba/resumen_integracion_contactos.xlsx'

# Definición campos para un contacto.
# Si alguno de estos está vacío, el contacto se considerará "incompleto".
CAMPOS_OBLIGATORIOS_CONTACTO = ['id_cliente', 'nombre_contacto', 'email_contacto']

In [64]:
print("--- INCIO EJERCICIO 2: Inicio de la Simulación de Integración API ---")
print("\n--- PASO 1: Configuración y Carga de Datos ---\n")

# 1.1: Cargar el DataFrame 'ventas_challenge_limpio_2.csv'
try:
    df_ejercicio2 = pd.read_csv(CSV_PARA_EJERCICIO2_PATH)
    print(f"'{CSV_PARA_EJERCICIO2_PATH}' cargado exitosamente. Filas: {df_ejercicio2.shape[0]}")
except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo '{CSV_PARA_EJERCICIO2_PATH}'. Asegurarse de que la ruta sea correcta.")
    exit() # Si el archivo no está, no se podria continuar.

# 1.2: Mostrar las primeras filas y tipos de datos para verificar que todo está Ok
print("\nPrimeras filas del DataFrame cargado (verificando 'id_cliente', 'nombre_contacto', 'email_contacto'):\n")
print(df_ejercicio2[['id_cliente', 'nombre_contacto', 'email_contacto']].head())
print("\nTipos de datos de las columnas clave:\n")
print(df_ejercicio2[['id_cliente', 'nombre_contacto', 'email_contacto']].dtypes)

--- INCIO EJERCICIO 2: Inicio de la Simulación de Integración API ---

--- PASO 1: Configuración y Carga de Datos ---

'/content/drive/MyDrive/Data_prueba/ventas_challenge_limpio_2.csv' cargado exitosamente. Filas: 800

Primeras filas del DataFrame cargado (verificando 'id_cliente', 'nombre_contacto', 'email_contacto'):

  id_cliente nombre_contacto             email_contacto
0      C0094   Cliente_C0094  contacto_2678@empresa.com
1      C0167   Cliente_C0167     contacto_8056@otra.com
2      C0029   Cliente_C0029  contacto_8612@empresa.com
3      C0052   Cliente_C0052     contacto_3517@otra.com
4      C0070   Cliente_C0070  contacto_5563@empresa.com

Tipos de datos de las columnas clave:

id_cliente         object
nombre_contacto    object
email_contacto     object
dtype: object


In [65]:
print("\n--- PASO 2: Definición de la 'API Ficticia' de Validación de Contactos---\n")

# Función ficticia para simular la validación de un contacto con una API externa
# Esta función simulará si un email ya existe en el CRM (True) o si es un nuevo contacto (False)
def simular_api_validar_contacto(email):
    """
    Simula una llamada a una API para validar si un email de contacto existe.
    Retorna True si el email contiene '@empresa.com', simulando un contacto existente.
    Retorna False si el email contiene '@otra.com' o es None/NaN, simulando un nuevo contacto.
    """
    if pd.isna(email): # Casos donde el email pueda ser NaN
        return False
    return '@empresa.com' in email

print("Función 'simular_api_validar_contacto' definida.")


--- PASO 2: Definición de la 'API Ficticia' de Validación de Contactos---

Función 'simular_api_validar_contacto' definida.


In [66]:
#Simulación de Validación y Clasificación de Contactos
#Se requirio usar la API ficticia para determinar qué clientes (del archivo CSV) son "contactos existentes" y cuáles son "nuevos contactos".

print("\n--- PASO 3: Simulación de Validación y Clasificación de Contactos (Parte A) ---\n")

# 3.1: Obtener la lista de CONTACTOS ÚNICOS para procesar

contactos_para_validar = df_ejercicio2[['id_cliente', 'nombre_contacto', 'email_contacto']].drop_duplicates().copy()

print(f"Se identificaron {contactos_para_validar.shape[0]} contactos únicos para validar.\n")

# 3.2: Aplicar la simulación de la API a cada contacto único
print("Aplicando la función de validación de API ficticia a cada email...")
contactos_para_validar['es_existente_en_crm'] = contactos_para_validar['email_contacto'].apply(simular_api_validar_contacto)

# 3.3: Clasificar los contactos basándose en el resultado de la validación
contactos_existentes_df = contactos_para_validar[contactos_para_validar['es_existente_en_crm'] == True]
nuevos_contactos_a_crear_df = contactos_para_validar[contactos_para_validar['es_existente_en_crm'] == False]

# 3.4: Contar los resultados para el resumen
total_contactos_existentes = len(contactos_existentes_df)
total_nuevos_contactos = len(nuevos_contactos_a_crear_df)

print(f"\nResultados de la Validación:")
print(f"   - Total de Contactos Únicos procesados: {len(contactos_para_validar)}")
print(f"   - Contactos simulados como EXISTENTES en el CRM: {total_contactos_existentes}")
print(f"   - Contactos simulados como NUEVOS para crear: {total_nuevos_contactos}")

print("\nPrimeros 5 contactos que YA EXISTEN en el CRM (simulado):\n")
print(contactos_existentes_df.head())
print("\nPrimeros 5 contactos NUEVOS para crear en el CRM (simulado):\n")
print(nuevos_contactos_a_crear_df.head())


--- PASO 3: Simulación de Validación y Clasificación de Contactos (Parte A) ---

Se identificaron 200 contactos únicos para validar.

Aplicando la función de validación de API ficticia a cada email...

Resultados de la Validación:
   - Total de Contactos Únicos procesados: 200
   - Contactos simulados como EXISTENTES en el CRM: 68
   - Contactos simulados como NUEVOS para crear: 132

Primeros 5 contactos que YA EXISTEN en el CRM (simulado):

  id_cliente nombre_contacto             email_contacto  es_existente_en_crm
0      C0094   Cliente_C0094  contacto_2678@empresa.com                 True
2      C0029   Cliente_C0029  contacto_8612@empresa.com                 True
4      C0070   Cliente_C0070  contacto_5563@empresa.com                 True
6      C0038   Cliente_C0038  contacto_3406@empresa.com                 True
8      C0026   Cliente_C0026  contacto_5208@empresa.com                 True

Primeros 5 contactos NUEVOS para crear en el CRM (simulado):

  id_cliente nombre_contacto

# **Generación JSON con los Nuevos Contactos "a Crear" (Parte B del Ejercicio)**

Crear un archivo en formato JSON que contenga solo los contactos que fueron clasificados como "nuevos" y que, teóricamente, serían enviados a la API del CRM para su creación.

In [67]:
print("\n--- PASO 4: Generando JSON con Nuevos Contactos---\n")

# 4.1: Preparación de la lista de diccionarios para el JSON
# Cada diccionario representará un contacto con sus propiedades clave.

lista_json_nuevos_contactos = []
for index, row in nuevos_contactos_a_crear_df.iterrows():
    # Solo añadimos al JSON si los campos principales no son nulos
    if pd.notna(row['id_cliente']) and pd.notna(row['email_contacto']):
        lista_json_nuevos_contactos.append({
            "id_cliente": row['id_cliente'],
            "nombre": row['nombre_contacto'],
            "email": row['email_contacto'],
             })

# 4.2: Guardar la lista de diccionarios como un archivo JSON
try:
    with open(JSON_NUEVOS_CONTACTOS_PATH, 'w', encoding='utf-8') as f:
        json.dump(lista_json_nuevos_contactos, f, ensure_ascii=False, indent=4)
    print(f"JSON con {len(lista_json_nuevos_contactos)} nuevos contactos guardado en: {JSON_NUEVOS_CONTACTOS_PATH}")
except Exception as e:
    print(f"ERROR: No se pudo guardar el archivo JSON. Detalles: {e}")


--- PASO 4: Generando JSON con Nuevos Contactos---

JSON con 58 nuevos contactos guardado en: /content/drive/MyDrive/Data_prueba/nuevos_contactos_a_crear.json


#**Creación Archivo Excel de Resumen (Parte C del Ejercicio)**

Generar un archivo Excel que consolide las métricas clave de este proceso de integración: total de nuevos contactos, contactos existentes y contactos con datos incompletos.

In [68]:
print("\n--- PASO 5: Archivo Excel ---\n-")

# 5.1: Identificación contactos con datos incompletos
# Uso de la lista de CAMPOS_OBLIGATORIOS_CONTACTO definida al inicio.
# Busqueda de nulos en *cualquiera* de estos campos dentro de contactos_para_validar.
contactos_con_datos_incompletos_df = contactos_para_validar[
    contactos_para_validar[CAMPOS_OBLIGATORIOS_CONTACTO].isnull().any(axis=1)
].copy()

total_contactos_incompletos = len(contactos_con_datos_incompletos_df)

print(f"Total de contactos con datos incompletos (basado en {', '.join(CAMPOS_OBLIGATORIOS_CONTACTO)}): {total_contactos_incompletos}")

# 5.2: Creación el DataFrame de resumen con las métricas clave
resumen_data = {
    'Métrica de Integración': [
        'Total de Contactos Únicos Procesados',
        'Contactos Simulados como Existentes en CRM',
        'Contactos Simulados como NUEVOS - Creados',
        'Contactos Identificados con Datos Incompletos'
    ],
    'Valor': [
        len(contactos_para_validar),
        total_contactos_existentes,
        total_nuevos_contactos,
        total_contactos_incompletos
    ]
}
df_resumen_final = pd.DataFrame(resumen_data)

print("\nDataFrame de Resumen Final:")
print(df_resumen_final)

# 5.3: Almacenamiento el DataFrame de resumen en un archivo Excel
try:
    df_resumen_final.to_excel(EXCEL_RESUMEN_PATH, index=False)
    print(f"\nResumen de la integración guardado exitosamente en: {EXCEL_RESUMEN_PATH}")
except Exception as e:
    print(f"ERROR: No se pudo guardar el archivo Excel de resumen. Detalles: {e}")

print("\n---Proceso de simulación de integración API COMPLETO ---")


--- PASO 5: Archivo Excel ---
-
Total de contactos con datos incompletos (basado en id_cliente, nombre_contacto, email_contacto): 74

DataFrame de Resumen Final:
                          Métrica de Integración  Valor
0           Total de Contactos Únicos Procesados    200
1     Contactos Simulados como Existentes en CRM     68
2      Contactos Simulados como NUEVOS - Creados    132
3  Contactos Identificados con Datos Incompletos     74

Resumen de la integración guardado exitosamente en: /content/drive/MyDrive/Data_prueba/resumen_integracion_contactos.xlsx

---Proceso de simulación de integración API COMPLETO ---
